
<br>
  - Advection-Diffusion-Reaction(u) = f    in the unit square<br>
            u = u_D  on the boundary<br>
No stabilization<br>


In [ ]:
# Install FEniCS
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl
    import dolfin
else:
    try:
        import ufl
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import ufl
        import dolfin

In [ ]:
from __future__ import print_function
from fenics import *
import matplotlib.pyplot as plt
import numpy as np

Create mesh and define function space

In [ ]:
mesh = UnitSquareMesh(16, 16)
V = FunctionSpace(mesh, 'P', 1)

In [ ]:
mu = Constant(1.0)
beta0 = 1000.0
beta1 = 1000.0
beta = Constant((beta0,beta1))
betamag = np.sqrt(beta0*beta0+beta1*beta1)

In [ ]:
h = mesh.hmax()

Define boundary condition<br>
_D = Expression('(exp(beta0/mu*x[0])-1)/(exp(beta0/mu)-1)*(exp(beta1/mu*x[1])-1)/(exp(beta1/mu)-1)', \<br>
 degree=3, beta0=beta0, beta1=beta1, mu=mu)

ex = Expression('(exp(beta0/mu*x[0])-1)/(exp(beta0/mu)-1)*(exp(beta1/mu*x[1])-1)/(exp(beta1/mu)-1)', \<br>
 degree=3, beta0=beta0, beta1=beta1, mu=mu)

xuex = Expression('beta0*(exp(beta0/mu*x[0]))/(exp(beta0/mu)-1)*(exp(beta1/mu*x[1])-1)/(exp(beta1/mu)-1)', \<br>
 degree=3, beta0=beta0, beta1=beta1, mu=mu)<br>
yuex = Expression('beta1*(exp(beta0/mu*x[0])-1)/(exp(beta0/mu)-1)*(exp(beta1/mu*x[1]))/(exp(beta1/mu)-1)', \<br>
 degree=3, beta0=beta0, beta1=beta1, mu=mu)

In [ ]:
tol = 1E-10

In [ ]:
def left(x, on_boundary):
    return on_boundary and x[0] <= tol

In [ ]:
def right(x, on_boundary):
    return on_boundary and x[0] >= 1.-tol

In [ ]:
def bottom(x, on_boundary):
    return on_boundary and x[1] <= tol

In [ ]:
def top(x, on_boundary):
    return on_boundary and x[1] >= 1.-tol

In [ ]:
bcl = DirichletBC(V, Constant(1.0), left)
bcr = DirichletBC(V, Constant(0.0), right)
bct = DirichletBC(V, Constant(0.0), top)
bcb = DirichletBC(V, Constant(1.0), bottom)

In [ ]:
bcs = [bcl,bcr,bct,bcb]

Define variational problem

In [ ]:
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(0.0)
#mu = Constant(1.0)
#beta0 = 10.0
#beta1 = 1000.0
#beta = Constant((10.0,1000.0))
delta = 1.0

In [ ]:
sigma =  Constant(0.0)
a = mu*dot(nabla_grad(u), nabla_grad(v))*dx + betamag*h/2*dot(nabla_grad(u), nabla_grad(v))*dx + \
    dot(beta,nabla_grad(u))*v*dx+sigma*u*v*dx # 
    
    
L = f*v*dx

quation = inner(nabla_grad(u), nabla_grad(v))*dx == f*v*dx

rm = parameters["krylov_solver"] # short form<br>
rm["absolute_tolerance"] = 1E-10<br>
rm["relative_tolerance"] = 1E-6<br>
rm["maximum_iterations"] = 1000<br>
et_log_level(DEBUG)

In [ ]:
set_log_level(LogLevel.ERROR)

Compute solution

In [ ]:
u = Function(V)
solve(a==L, u, bcs)#, solver_parameters={"linear_solver": "cg", "preconditioner": "ilu"})
info(parameters,True)

Save solution to file in VTK format

In [ ]:
vtkfile = File('adr/HandsOn.pvd')
vtkfile << u